## Notebook Overview

This notebook builds upon the feature-engineered dataset from the previous notebook (`04_feature_engineering.ipynb`) and focuses on **Model Training and Evaluation**. Our primary goal is to develop a credit risk prediction model that excels at identifying potential loan defaulters, thereby minimizing financial losses for retail banks while also considering their desired balance between risk aversion and loan approval rates. This translates to maximizing the recall of the positive class (loan defaulters) while maintaining acceptable precision and overall model performance.

### 0.5.1 Objectives

The main objectives of this notebook are:

1. **Model Selection:** Choose algorithms suitable for imbalanced classification problems.
2. **Model Training:** Train models with a focus on identifying potential defaulters.
3. **Hyperparameter Tuning:** Optimize models to increase recall for the positive class.
4. **Model Evaluation:** Assess models primarily on recall, while considering precision, F2-score, AUC-PR, and overall performance.
5. **Model Comparison:** Compare different models based on their ability to identify true positives and balance the precision-recall trade-off.
6. **Threshold Adjustment:** Explore the impact of classification thresholds on recall and precision, collaborating with retail banks to determine the optimal threshold.

### 0.5.2 Importance of Focusing on Recall

Prioritizing recall for defaulter prediction is crucial for minimizing financial losses, which is the primary business objective in credit risk assessment. The cost of missing a potential defaulter (false negative) is typically much higher than the cost of incorrectly classifying a non-defaulter as high-risk (false positive). While we prioritize recall, we will also carefully consider the precision-recall trade-off and aim for a model that maximizes recall without severely impacting precision. Techniques like threshold adjustment and cost-sensitive learning will be used to balance these metrics effectively. Furthermore, demonstrating a thorough approach to risk identification aligns with regulatory expectations in the financial sector, supporting the banks' compliance needs. This approach also allows for more conservative lending practices, which can be adjusted based on the bank's specific risk tolerance.

### 0.5.3 Our Approach

In this notebook, we will focus on the following modeling tasks:

1. **Data Preparation:** Address class imbalance using techniques like SMOTE or class weighting.
2. **Baseline Model:** A logistic regression model with class weights inversely proportional to class frequencies will serve as our baseline. This will provide a benchmark for evaluating more complex models.
3. **Advanced Models:** Train and evaluate models known for handling imbalanced data:
   - Decision Trees with adjusted class weights
   - Random Forest with balanced class weights
   - Gradient Boosting (XGBoost, LightGBM) with `scale_pos_weight` adjustment
4. **Hyperparameter Tuning:** We will employ techniques like GridSearchCV or RandomizedSearchCV, optimizing for the F2-score (which gives more weight to recall) or a custom cost-sensitive scoring function.
5. **Model Evaluation:** Prioritize recall in our metrics, while also considering precision, F2-score, AUC-PR, and AUC-ROC.
6. **Threshold Adjustment:** We will experiment with different classification thresholds and work closely with retail banks to determine the optimal threshold that balances their desired level of risk aversion with acceptable loan approval rates.
7. **Ensemble Methods:** Explore ensemble techniques that can improve recall without severely impacting precision.
8. **Cost-Sensitive Learning:** Incorporate misclassification costs to reflect the higher cost of false negatives, aligning the model's objective with the business goal of minimizing financial losses.

By the end of this notebook, we aim to have a model (or ensemble of models) that excels at identifying potential loan defaulters, providing the bank with a powerful tool for risk assessment and mitigation.


In [8]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.preprocessing import FunctionTransformer
from lightgbm.callback import early_stopping

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
)
from sklearn.model_selection import cross_val_predict

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    precision_recall_curve,
    auc,
    confusion_matrix,
    f1_score,
    recall_score,
    precision_score,
    make_scorer,
)
from sklearn.utils.class_weight import compute_class_weight

from retail_bank_risk.model_training_utils import downscale_dtypes
from retail_bank_risk.advanced_visualizations_utils import (
    plot_confusion_matrix,
    plot_model_performance,
    shap_summary_plot,
    shap_force_plot,
    plot_roc_curve,
    plot_precision_recall_curve,
    plot_combined_confusion_matrices,
    plot_learning_curve,
)

from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel


In [2]:
train_df = pd.read_parquet("../data/processed/application_train_engineered.parquet")
test_df = pd.read_parquet("../data/processed/application_test_engineered.parquet")

print(f"Training Data Shape: {train_df.shape}")
print(f"Test Data Shape: {test_df.shape}")

Training Data Shape: (307511, 78)
Test Data Shape: (48744, 77)


In [3]:
train_df, test_df = downscale_dtypes(train_df, test_df, target_column='target')

train_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 78 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   reg_city_not_work_city_0                 307511 non-null  uint8  
 1   reg_city_not_work_city_1                 307511 non-null  uint8  
 2   region_rating_client_w_city              307511 non-null  float32
 3   region_rating_client                     307511 non-null  float32
 4   name_contract_type_cash loans            307511 non-null  uint8  
 5   name_contract_type_revolving loans       307511 non-null  uint8  
 6   code_gender_m                            307511 non-null  uint8  
 7   code_gender_f                            307511 non-null  uint8  
 8   flag_own_car_n                           307511 non-null  uint8  
 9   flag_own_car_y                           307511 non-null  uint8  
 10  flag_own_realty_y               

In [4]:
X = train_df.drop(["target", "sk_id_curr"], axis=1)
y = train_df["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_test = test_df.drop("sk_id_curr", axis=1)
sk_id_curr = test_df["sk_id_curr"]

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

Training set shape: (246008, 76)
Validation set shape: (61503, 76)
Test set shape: (48744, 76)


In [5]:
def sanitize_feature_names(X):
    return X.rename(columns=lambda x: re.sub(r'[^\w]+', '_', x))
sanitize_transformer = FunctionTransformer(sanitize_feature_names)

In [6]:
pipelines = {
    'Dummy Classifier': Pipeline([
        ('sanitizer', FunctionTransformer(sanitize_feature_names)),
        ('classifier', DummyClassifier(strategy='stratified', random_state=42))
    ]),
    'Logistic Regression': Pipeline([
        ('sanitizer', FunctionTransformer(sanitize_feature_names)),
        ('scaler', StandardScaler()),
        ('feature_selection', SelectFromModel(LogisticRegression(random_state=42))),
        ('classifier', LogisticRegression(random_state=42, class_weight='balanced',
                                          max_iter=1000, penalty='l2', C=0.1))
    ]),
    'Decision Tree': Pipeline([
        ('sanitizer', FunctionTransformer(sanitize_feature_names)),
        ('feature_selection', SelectFromModel(DecisionTreeClassifier(random_state=42))),
        ('classifier', DecisionTreeClassifier(random_state=42, class_weight='balanced',
                                              max_depth=3, min_samples_split=5))
    ]),
    'Random Forest': Pipeline([
        ('sanitizer', FunctionTransformer(sanitize_feature_names)),
        ('feature_selection', SelectFromModel(RandomForestClassifier(random_state=42))),
        ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced',
                                              n_jobs=1, max_depth=5, n_estimators=100,
                                              min_samples_split=5, bootstrap=True))
    ]),
    'Gradient Boosting': Pipeline([
        ('sanitizer', FunctionTransformer(sanitize_feature_names)),
        ('feature_selection', SelectFromModel(GradientBoostingClassifier(random_state=42))),
        ('classifier', GradientBoostingClassifier(random_state=42, max_depth=3,
                                                  n_estimators=100, learning_rate=0.01,
                                                  subsample=0.8, min_samples_split=5))
    ]),
    'XGBoost': Pipeline([
        ('sanitizer', FunctionTransformer(sanitize_feature_names)),
        ('feature_selection', SelectFromModel(xgb.XGBClassifier(random_state=42))),
        ('classifier', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss',
                                         random_state=42, scale_pos_weight=len(y)/sum(y),
                                         max_depth=3, n_estimators=100, learning_rate=0.01,
                                         subsample=0.8, colsample_bytree=0.8,
                                         min_child_weight=5, n_jobs=1))
    ]),
    'LightGBM': Pipeline([
        ('sanitizer', FunctionTransformer(sanitize_feature_names)),
        ('feature_selection', SelectFromModel(lgb.LGBMClassifier(random_state=42))),
        ('classifier', lgb.LGBMClassifier(random_state=42, class_weight='balanced',
                                          max_depth=3, n_estimators=100, learning_rate=0.01,
                                          subsample=0.8, colsample_bytree=0.8,
                                          min_child_samples=5, n_jobs=1))
    ])
}

In [9]:
def evaluate_model(name, model, X, y):
    print(f"Evaluating {name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Predictions
    y_pred = cross_val_predict(model, X, y, cv=cv, method='predict')
    y_pred_proba = cross_val_predict(model, X, y, cv=cv, method='predict_proba')[:, 1]

    # Metrics
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    auc_roc = roc_auc_score(y, y_pred_proba)

    print(f"{name} Cross-validation results:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print("=" * 60 + "\n")

    return {
        'model': name,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc
    }

# Evaluate models
results = []
for name, pipeline in pipelines.items():
    result = evaluate_model(name, pipeline, X, y)
    results.append(result)

# Sort and print final ranking
print("Model Performance Ranking:")
for metric in ['precision', 'recall', 'f1_score', 'auc_roc']:
    print(f"\nRanking by {metric}:")
    sorted_results = sorted(results, key=lambda x: x[metric], reverse=True)
    for i, result in enumerate(sorted_results, 1):
        print(f"{i}. {result['model']}: {metric} = {result[metric]:.4f}")

Evaluating Dummy Classifier...
Dummy Classifier Cross-validation results:
Precision: 0.0831
Recall: 0.0822
F1-Score: 0.0826
AUC-ROC: 0.5013

Evaluating Logistic Regression...
Logistic Regression Cross-validation results:
Precision: 0.1566
Recall: 0.6663
F1-Score: 0.2536
AUC-ROC: 0.7372

Evaluating Decision Tree...
Decision Tree Cross-validation results:
Precision: 0.2039
Recall: 0.6926
F1-Score: 0.3151
AUC-ROC: 0.7981

Evaluating Random Forest...
Random Forest Cross-validation results:
Precision: 0.2550
Recall: 0.7633
F1-Score: 0.3822
AUC-ROC: 0.8873

Evaluating Gradient Boosting...
Gradient Boosting Cross-validation results:
Precision: 1.0000
Recall: 0.1291
F1-Score: 0.2287
AUC-ROC: 0.8503

Evaluating XGBoost...
XGBoost Cross-validation results:
Precision: 0.3213
Recall: 0.8750
F1-Score: 0.4700
AUC-ROC: 0.9448

Evaluating LightGBM...
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing

We evaluated several machine learning models for credit risk prediction, with a focus on maximizing **recall** to minimize false negatives in identifying potential loan defaulters.

**XGBoost** emerged as the top-performing model, achieving the highest recall (**0.8750**), F1-score (**0.4700**), and AUC-ROC (**0.9448**), closely followed by **LightGBM** with similar strong performance across these metrics (recall: **0.8415**, F1-score: **0.4383**, AUC-ROC: **0.9262**).

**Random Forest** also showed promising results, while **Gradient Boosting** achieved perfect precision (**1.0000**) but at the cost of very low recall (**0.1291**), indicating potential overfitting.

The performance of these advanced models significantly outpaced simpler approaches like Decision Trees and Logistic Regression, as well as the baseline Dummy Classifier.

Based on these results, we recommend focusing further optimization efforts on **XGBoost** and **LightGBM** models, with potential exploration of ensemble methods combining these top performers to further enhance predictive accuracy in credit risk assessment.